In [1]:
import os
import cv2 as cv
import numpy as np
from PIL import Image
import torch
from torch import nn
from skorch import NeuralNetClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
device = torch.device(
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

In [3]:
def load_images_from_folder(folder):
    images = []
    for file in os.listdir(folder):
        img = cv.imread(os.path.join(folder,file))
        if(img is not None):
            images.append(img)
    return images

In [4]:
def resize_and_flatten(img, target_size=(64,64)):
    resized_img = cv.resize(img, target_size)
    flattened_img = resized_img.reshape(target_size[0] * target_size[1] * 3)
    return flattened_img

In [5]:
cat_data = []
for img in load_images_from_folder("../notebooks/myfirstNN/data/cat"):
    cat_data.append(resize_and_flatten(img))
dog_data = []
for img in load_images_from_folder("../notebooks/myfirstNN/data/dog"):
    dog_data.append(resize_and_flatten(img))

In [6]:
print(str(len(cat_data)) + " cat images")
print(str(len(dog_data)) + " dog images")

772 cat images
772 dog images


In [7]:
X = []
y = []

for entity in cat_data:
    X.append(entity)
    y.append(1)

for entity in dog_data:
    X.append(entity)
    y.append(0)


In [8]:
class MyNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(64*64*3, 16*16),
            nn.ReLU(),
            nn.Linear(16*16, 16*16),
            nn.ReLU(),
            nn.Linear(16*16, 2)
        )
    def forward(self, x, **kwargs):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [9]:
net = NeuralNetClassifier(
    module = MyNN,
    max_epochs=10,
    lr=0.001,
    iterator_train__shuffle=True,
)

In [10]:
params = {    
    'lr': [0.001, 0.01, 0.0001],
    'max_epochs': [1,2,5, 10, 20, 25],
}

gs = GridSearchCV(net, params, cv=2)
gs.fit(torch.FloatTensor(X), torch.LongTensor(y))
pd.DataFrame(gs.cv_results_)

C:\Users\GeorgBecker\AppData\Local\Temp\ipykernel_27128\3442304046.py:7: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:264.)
  gs.fit(torch.FloatTensor(X), torch.LongTensor(y))


  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1           nan       0.4968           nan  0.1800
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1           nan       0.5032           nan  0.1790
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1           nan       0.5032           nan  0.1621
      2           nan       0.5032           nan  0.1549
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1           nan       0.4968           nan  0.1571
      2           nan       0.4968           nan  0.1625
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1           nan       0.5032           nan  0.1595
      2           nan       0.5

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_lr,param_max_epochs,params,split0_test_score,split1_test_score,mean_test_score,std_test_score,rank_test_score
0,0.457771,0.241785,0.098959,0.001439,0.001,1,"{'lr': 0.001, 'max_epochs': 1}",0.500000,0.5,0.500000,0.000000,2
1,0.356276,0.001304,0.103435,0.001096,0.001,2,"{'lr': 0.001, 'max_epochs': 2}",0.500000,0.5,0.500000,0.000000,2
2,0.886866,0.015064,0.106535,0.002005,0.001,5,"{'lr': 0.001, 'max_epochs': 5}",0.500000,0.5,0.500000,0.000000,2
3,1.726296,0.027418,0.098342,0.005811,0.001,10,"{'lr': 0.001, 'max_epochs': 10}",0.500000,0.5,0.500000,0.000000,2
4,3.354784,0.018467,0.101902,0.004625,0.001,20,"{'lr': 0.001, 'max_epochs': 20}",0.500000,0.5,0.500000,0.000000,2
5,4.215923,0.005524,0.098032,0.003500,0.001,25,"{'lr': 0.001, 'max_epochs': 25}",0.500000,0.5,0.500000,0.000000,2
6,0.192551,0.005495,0.096524,0.003998,0.01,1,"{'lr': 0.01, 'max_epochs': 1}",0.500000,0.5,0.500000,0.000000,2
7,0.368575,0.003003,0.100285,0.003753,0.01,2,"{'lr': 0.01, 'max_epochs': 2}",0.500000,0.5,0.500000,0.000000,2
8,0.875901,0.012522,0.109518,0.000003,0.01,5,"{'lr': 0.01, 'max_epochs': 5}",0.500000,0.5,0.500000,0.000000,2
9,1.729139,0.029986,0.102025,0.003502,0.01,10,"{'lr': 0.01, 'max_epochs': 10}",0.500000,0.5,0.500000,0.000000,2


In [11]:
test_imgs = load_images_from_folder("../notebooks/myfirstNN/data/cat")